# Adding All

In [213]:
macro select_force(type)
    # Parameters
    μ = (:μ)
    r_max = (:r_max)
    r_min = (:r_min)
    α = (:α)

    # List of forces functions
    if type == (:cubic)
        struct_select = quote
            Base.@kwdef struct $type
                μ::Float64
                r_max::Float64
                r_min::Float64
                f::Function = r -> ifelse.(
                    r.<=$r_max, 
                    $μ .* (r .- r_max).^2 .* (r .- r_min), 
                    0.0
                )
            end
        end
    elseif type == (:GLS)
        struct_select = quote
            Base.@kwdef struct $type
                μ::Float64
                r_max::Float64
                r_min::Float64
                α :: Float64
                f::Function = r -> ifelse.(
                    r.<=$r_min,  
                    $μ .* log(1 .+ r .- $r_min),
                    ifelse.(
                        r.<=$r_max,
                        $μ .* (r .- $r_min) .* exp.(-α .* (r .- r_min)),
                        0 
                    )
                )
            end
        end
    else
        struct_select = quote
            println("FORCE FUNCTION NOT FOUNDED")
        end
    end
    return esc(:($struct_select))
end

@select_force (macro with 1 method)

## ERROR

In [214]:
@select_force ESTE_NO

FORCE FUNCTION NOT FOUNDED


## Cubic

In [215]:
@select_force cubic
force = cubic(μ = 1, r_max = 2, r_min = 1)
dump(force)

cubic
  μ: Float64 1.0
  r_max: Float64 2.0
  r_min: Float64 1.0
  f: #186 (function of type var"#186#188"{Int64, Int64, Int64})
    μ: Int64 1
    r_max: Int64 2
    r_min: Int64 1


In [216]:
force.f(1.2)

0.128

In [217]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Float64}:
 0.0747417  0.0209309  0.111782
 0.0        0.0        0.00420164
 0.0        0.0        0.0324015

## GLS

In [218]:
@select_force GLS
force = GLS(μ = 1, r_max = 2, r_min = 1, α = 2)
dump(force)

GLS
  μ: Float64 1.0
  r_max: Float64 2.0
  r_min: Float64 1.0
  α: Float64 2.0
  f: #191 (function of type var"#191#193"{Int64, Int64, Int64, Int64})
    μ: Int64 1
    r_max: Int64 2
    r_min: Int64 1
    α: Int64 2


In [219]:
force.f(1.2)

0.13406400920712783

In [220]:
force.f(2 .* rand(3,3) .+ 1)

3×3 Matrix{Real}:
 0  0.183515  0
 0  0         0.173453
 0  0.119121  0.145331